# Imports

In [ ]:
import pandas as pd
from asapdiscovery.data.services.fragalysis.fragalysis_reader import FragalysisFactory
from pathlib import Path

In [ ]:
xseries_dir = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/mpro_fragalysis-04-01-24_x_series")

In [ ]:
sf = FragalysisFactory(parent_dir=xseries_dir)

In [ ]:
complexes = sf.load()

In [ ]:
len(complexes)

In [ ]:
metadata = pd.read_csv(xseries_dir / "metadata.csv", index_col=0)

In [ ]:
xseries = metadata[(metadata["crystal_name"].str.contains("-x"))]

In [ ]:
xseries.site_name.unique()

In [ ]:
bad_labels = ["XChem screen - surface", "Moonshot - other active site", "XChem screen - dimer interface", "XChem Screen - xtal contact", "XChem Screen - Covalent hits", "Moonshot - allosteric"]

In [ ]:
xactive = xseries[~xseries.site_name.isin(bad_labels)]

In [ ]:
filtered_complexes = [_complex for _complex in complexes if _complex.ligand.compound_name in xactive.alternate_name.tolist()]

In [ ]:
len(filtered_complexes)

In [ ]:
structure_names = [c.target.target_name for c in filtered_complexes]

In [ ]:
active_site_dir = xseries_dir.parent / "mpro_fragalysis-04-01-24_x_series_active_site/aligned"
active_site_dir.mkdir(exist_ok=True)

In [ ]:
import shutil
from tqdm import tqdm

In [ ]:
source_path = xseries_dir / "aligned"
source_dirs = [fp for fp in source_path.glob("*") if fp.is_dir()]
for fp in tqdm(source_dirs):
    if not fp.is_dir():
        continue
    if fp.name in structure_names:
        newpath = active_site_dir / fp.name
        shutil.copytree(fp, newpath)